In [1]:
from pyomo.environ import *
import numpy as np
import matplotlib.pyplot as plt
import bisect
from pyomo.opt import SolverStatus, TerminationCondition

In [2]:
def nc_underest(new_lb, new_ub, v_list, model_list, add_node_num=0, n_samples=5, tolerance=1e-8):
    """
    Parameters:
        delta (float): range of variable
        n_samples (float): number of sample nodes
        v_list (list): list of scenario functions  
        model: model with submodels corresponds to each scenario
        tolerance (float): decide when to stop

    Returns: delta (float): delta
             errors (float): hausdorff error
             y_nodes (list): y node (to make plot)
             as_nodes_list[0] (list): As node value (to make plot)
             ms_list[0] (float): ms for first scenario (to make plot)
    """
    N = len(model_list) 
    #N=1 ###############################################################################################
    y_nodes = np.linspace(new_lb, new_ub, n_samples).tolist()
    lst = np.ones(N).tolist() 
    y_opt_list = np.ones(N).tolist() 
    as_nodes_list = np.ones(N).tolist() 
    ms_list = np.ones(N).tolist() 
    
    current_new_nodes = []
    new_nodes_list = []
    
    #'''
    solver = SolverFactory('gurobi')
    solver.options['FeasibilityTol'] = 1e-9
    solver.options['OptimalityTol'] = tolerance
    solver.options['NonConvex'] = 2 
    #'''
    '''
    solver = SolverFactory('scip', executable='/Users/yinhuang/miniconda3/bin/scip')
    solver.options['numerics/feastol'] = tolerance
    solver.options['numerics/dualfeastol'] = tolerance
    solver.options['limits/gap'] = tolerance
    '''
        
    # reset bounds
    for i in range(N):
        model_list[i].y.setlb(new_lb)
        model_list[i].y.setub(new_ub)
    
    for k in range(add_node_num+1):
        for i in range(N):
            print('add node ',k)
            print('model ',i)
            
            as_nodes_list[i] = [v_list[i](y) for y in y_nodes]
        
            # define piecewise function for each scenario
            if hasattr(model_list[i], 'As'):
                model_list[i].del_component('As')
            if hasattr(model_list[i], 'pw'):
                model_list[i].del_component('pw')
            if hasattr(model_list[i], 'obj'):
                model_list[i].del_component('obj')

            model_list[i].As = Var()
            model_list[i].pw = Piecewise(
                model_list[i].As, model_list[i].y,
                pw_pts=y_nodes,
                f_rule=as_nodes_list[i],
                pw_constr_type='EQ',
                pw_repn='SOS2'
            )

            model_list[i].obj = Objective(expr=model_list[i].obj_expr - model_list[i].As, sense=minimize)
            results = solver.solve(model_list[i])
            ms_list[i] = results.problem.lower_bound
            
            new_node = value(model_list[i].y)
            if (new_node not in y_nodes) and (k != add_node_num):
                bisect.insort(y_nodes, new_node)
            #y_opt_list[i] = value(model_list[i].y)
            

    print()
    
    # define and solve the sum model
    #print(22)
    #assum_nodes = np.sum(as_nodes_list, axis=0).tolist()
    print('abab',len(y_nodes))
    print('aa',len(as_nodes_list[0]))
    assum_nodes = np.sum(as_nodes_list, axis=0)
    print('bb',len(as_nodes_list[0]))
    model_sum = ConcreteModel()
    model_sum.y = Var(bounds=(new_lb, new_ub))
    model_sum.As = Var()
    model_sum.pw = Piecewise(
        model_sum.As, model_sum.y,
        pw_pts=y_nodes,
        f_rule=list(assum_nodes),
        pw_constr_type='EQ',
        pw_repn='SOS2'
    )
    model_sum.obj = Objective(expr = model_sum.As, sense=minimize)
    solver.solve(model_sum)
    results = solver.solve(model_sum)
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        pass
    else:
        print("bad bad bad")
    # get the output
    errors = results.problem.lower_bound + sum(ms_list)
    #errors = value(model_sum.obj) + sum(ms_list)
    errors = -errors
    
    errors = -ms_list[0]
    delta = (new_ub-new_lb)/2
    
    print('node is ',add_node_num)
    print('error is ',errors)
    print('')
    print(' ')
    
    #return delta, errors, 1, 2  # diameter, error
    return delta, errors, y_nodes, as_nodes_list[0], ms_list[0], y_opt_list[0]

In [3]:
from pyomo.environ import *
#import math

# 创建模型
m1 = ConcreteModel()
m1.x1 = Var(bounds=(-10, 10)) 
m1.y = Var(bounds=(-10, 10))




# 定义目标函数 f(x, y) = y^2 + 5*sin(x)
m1.obj = Objective(
    expr = m1.y**2 + 5 * sin(m1.x1),
    sense = minimize
)

m1.c1 = Constraint(expr = m.y >= 0.5*m.x + 1)
m.c2 = Constraint(expr = m.x**2 + m.y**2 <= 25)

# 求解
SolverFactory('ipopt').solve(m, tee=True)

# 输出结果
print("x =", value(m.x))
print("y =", value(m.y))
print("f(x,y) =", value(m.obj))


NameError: name 'm' is not defined

In [ ]:
# build function for each scenario 
def v_1(y):  
    if y < 5 or y > 20:
        return float('inf')
    elif 5 <= y < 6.53:
        return 2.5 * y - 10 * (2 * y - 10)
    else:
        return 


def v_2(y): 
    if y < 2.22 or y > 20:
        return float('inf')
    elif 2.22 <= y < 3.49:
        return 2.5 * y - 20 * (4.5 * y - 10)
    else:
        return 2.5 * y - 20 * 20 / y

v_list = [v_1, v_2]

# set up parameters and solvers
n_samples = 10
add_node_num=5
ylb = 6
yub = 15
tolerance=1e-9

m1 = ConcreteModel()
m1.y = Var(bounds=(ylb, yub))
m1.x1 = Var(bounds=(0, None))        ##################################
m1.c1 = Constraint(expr= 2*m1.y-m1.x1>=10)
m1.c2 = Constraint(expr= m1.x1 * m1.y<=20)
#m1.c3 = Constraint(expr= 2*sqrt(m1.x1+5)*log(m1.x1+2) <= 35)
m1.obj_expr = Expression(expr=2.5*m1.y - 10*m1.x1)